In [37]:
from opik import configure
from opik.integrations.langchain import OpikTracer

# configure(api_key="dHhF5jDOpifAMNJH9UalSjR0o", workspace="yu-li")
# opik_tracer = OpikTracer(project_name="sku_match")


OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".
OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/liyu/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [38]:
# 使用langgraph接入大模型，使用大模型判断是否相似
# 输入商品名称和top3商品名称，输入相似的结果
from langchain_openai import ChatOpenAI
import os

# os.environ["LANGSMITH_API_KEY"] = "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"
# os.environ["LANGSMITH_PROJECT"] = "rag"
# os.environ["LANGSMITH_TRACING"] = "true"


qwen3_8B = ChatOpenAI(
    model="Qwen/Qwen3-8B",
    base_url="https://api.siliconflow.cn/v1",
    api_key="sk-idaudysppselrwglygkbtatkregsbxhaxypaeulbfpavrals",
)


In [39]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict


class RankSelect(BaseModel):
    """Model for match top rank. Provide json constraints."""

    match_index: int = Field(
        description="返回origin_product描述一致的候选商品索引，没有候选商品匹配或其他情况返回0，返回值和分析过程保持一致。",
    )
    match_reason: str = Field(
        description="返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。",
    )

In [40]:

rank_prompt = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为候选匹配商品信息。
2. 每个商品信息包含品牌、商品名、规格数量3部分信息，分析时先提取 origin_product 输入商品的品牌、商品名、规格数量信息。
3. 提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、商品名、规格数量信息，判断是否为同款商品。
4. 根据匹配规则选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

<Information Extraction Rules>
以下为信息提取规则：
1.商品品牌：通常位于信息前部，一般使用空格分离，部分商品可能不含品牌名。如：旺仔 精选进口乳源原味小馒头 14g_袋，品牌提取为旺仔；部分品牌和商品名重叠，如 元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，品牌为元气森林；
2.商品名称：通常位于信息中部，需要去除描述部分。如：元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，名称提取为青柠口味电解质饮料；
3.商品规格数量：
    1）通常位于信息尾部，由规格和数量两部分组成。如：喜力Heineken 11.4°P啤酒 500ml3罐_包，规格为500ml3罐_包，数量为1；
    2）部分商品数量信息位于尾部且只有数字，如：乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货），规格数量部分为500ml_听3，规格为500ml_听，数量为3；
    3）如果没有数量部分默认为1，如：旺仔 精选进口乳源原味小馒头 14g_袋，规格为14g_袋，数量为1；
</Information Extraction Rules>

<Match rules>
以下是origin_product和候选商品的匹配规则：
1. 商品品牌不一致时视为不匹配，例如origin_product识别品牌为可口，candidate识别为品牌为百事；特别注意，评判时由于提取品牌可能包含商品信息，因此商品部分词汇重叠也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量和规格严格一致才视为匹配，数据和规格不一致标明不是同款商品，匹配度为0。以下是规格数据匹配时处理注意事项：
    1）规格可包括L、kg、千克、盒、件、片、条等市面计量单位，评判时注意规格单位换算，如1L和1000ml为一致；
    2）个、袋、盒、瓶等单个包装规格在整体语义下视为同等规格；
    3）规格中没有体现数量，视为数量为1。
    4）比较商品双方规格数量都不存在视为一致。
3. 商品名称语义不一致视为不匹配，origin_product识别为识字卡，candidate识别为挪车卡。
4. 候选top_candidate都不匹配时不需要寻找最接近的商品，视为没有商品匹配，返回0。
</Match rules>


以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回origin_product描述一致的候选商品索引；没有候选商品匹配或其他情况返回0。
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>

<Critical Reminder>
1. 特别注意，如果候选商品信息都不是同款，不需要考虑寻找最接近的候选商品，说明没有同款商品，直接返回0。
2. 必须保证分析结果编号或索引与返回的match_index字段值相同，分析得出没有匹配结果时返回0。
</Critical Reminder>
"""

In [6]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict


class MatchSelect(BaseModel):
    """Model for match top rank. Provide json constraints."""

    match_index: int = Field(
        description="返回origin_product描述一致的候选商品索引。如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；没有候选商品匹配或其他情况返回0。",
    )


rank_prompt_no_reason = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品类型的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
2. 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
3. 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
4. 依次提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、描述、商品名、规格数量信息，判断是否为同款商品。
5. 全面分析思考，选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

<Guidelines>
以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品为不同商品：
1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时由于提取品牌可能包含商品信息，因此商品部分词汇重叠也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，规格数量需要同时满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位；评判时注意规格单位换算，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下视为同等规格。
3. 商品名语义或分类不一致，origin_product识别为识字卡，candidate识别为挪车卡。
4. 商品同类型描述完全不一致，origin_product提取描述为橘子味，candidate提取描述为葡萄味。
</Guidelines>

<Examples>
示例1：
origin_product：卫龙魔芋爽 微辣麻酱素毛肚 （15+3）g_袋
top1_candidate：卫龙 魔芋爽 香辣味素毛肚 15克_袋
分析：origin_product和top1_candidate品牌、商品名都为卫龙和魔芋爽，但是由于规格数量不同，origin_product为18g/袋,top1_candidate为15g/袋，所以不是同款商品。

示例2：
origin_product：【毛绒长款睡袍】卡皮巴拉冬季新款加厚睡袍卡通可爱风珊瑚绒睡衣女家居服_件
top1_candidate：珊瑚绒睡袍女冬季甜美加厚新款长款睡裙法兰绒睡衣女可外穿家居服_件（没有裤子）
分析：origin_product和top1_candidate品牌、商品无法提取，商品描述未高度重叠，所有判断不是同款商品，需要返回0，match_index=0。
</Examples>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Output Format>
请严格按照以下键值以有效的 JSON 格式进行返回：
"match_index": int. 返回origin_product描述一致的候选商品索引。 如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；没有候选商品匹配或其他情况返回0。
</Output Format>

<Critical Reminder>
- 规格数量未必须条件，需满足完全一致，基本一致视为不匹配。
- 当商品提取不到品牌、数量规格信息时，商品名称和商品描述需要高度一致才视为同一款商品。
- 特别注意，如果候选商品信息都不是同款，不需要考虑寻找最接近的候选商品，说明没有同款商品，直接返回0。
- 必须保证分析结果编号或索引与返回的match_index字段值相同，分析得出没有匹配结果时返回0。
</<Critical Reminder>
"""

In [41]:
from LLM import longcat_flash_chat, GLM4_9B, qwen3_14b, glm4_5_air, qwen3_next_80b, qwen3_30b_instruct, glm_z1_9b, \
    qwen3_235b_thinking, qwen3_30b_thinking, qwen3_235b_instruct, qwen3_max

model_qwen_8b = qwen3_8B.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_glm_9b = GLM4_9B.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_glm_9b_no_reason = GLM4_9B.with_structured_output(MatchSelect).with_retry(stop_after_attempt=3)
model_glm_z1_9b = glm_z1_9b.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_14b = qwen3_14b.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_30b_thinking = qwen3_30b_thinking.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_30b = qwen3_30b_instruct.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_30b_no_reason = qwen3_30b_instruct.with_structured_output(MatchSelect).with_retry(stop_after_attempt=3)
model_qwen3_next_80b = qwen3_next_80b.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_glm4_5_air = glm4_5_air.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_235b_thinking = qwen3_235b_thinking.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_235b_instruct = qwen3_235b_instruct.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_max = qwen3_max.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_longcat_flash = longcat_flash_chat.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)


In [6]:
# origin_product = "达利园 香奶味软面包  200g／袋 面包就要达利园"
# top1_candidate="达利园 香奶味法式软面包 360g/袋"
# top2_candidate="达利园 桂圆莲子八宝粥 360g/罐"
# top3_candidate="洽洽 香瓜子 160g/袋"
# top1_candidate = ""
# top2_candidate = "达利园 香奶味法式软面包 360g/袋"
# top3_candidate = ""

In [22]:
origin_product = "东鹏特饮 东鹏特饮维生素功能饮料 500ml_瓶"
top1_candidate = "东鹏  维生素能量饮料 500ml"
top2_candidate = ""
top3_candidate = ""

In [9]:
origin_product = "迪味臻 五香卤蛋 30g_袋"
top1_candidate = "迪味臻 五香卤蛋 30g_个"
top2_candidate = ""
top3_candidate = ""

In [37]:
origin_product = "乌苏啤酒 11°P红乌苏啤酒 620ml_瓶"
top1_candidate = "乌苏啤酒 11°P夺命大乌苏啤酒 620ml_瓶"
top2_candidate = ""
top3_candidate = ""

In [ ]:
origin_product = "禛香 禛香肥牛味大豆素肉大豆粕膨化制品 18g_袋"
top1_candidate = "禛香 牛肉味素肉辣条 儿时怀旧解馋小零食小吃休闲食品辣条肥牛味 18g_袋"
top2_candidate = ""
top3_candidate = ""

In [43]:
origin_product = "苏菲SOFY M码超薄款超熟睡安心裤裤型卫生巾 2条_包"
top1_candidate = "【5包特惠】苏菲SOFY M码超薄款超熟睡安心裤裤型卫生巾 2条_包5"
top2_candidate = "苏菲 XL码超薄款超熟睡安心裤 2条5包"
top3_candidate = "【苏菲】超熟睡安心裤裤型卫生巾M号 2条_袋"

In [7]:
origin_product = "娃哈哈AD钙奶儿童奶含乳饮料饮品童年哇哈哈 220gx4瓶／排"
top1_candidate = "【2排】娃哈哈 AD钙奶饮料 220g4瓶2排"
top2_candidate = "【4瓶装】娃哈哈 AD钙奶饮料 220g4瓶_排"
top3_candidate = "【整箱】娃哈哈 AD钙奶饮料 220g24瓶_箱"

In [42]:
origin_product = "康师傅 冰力十足冰红茶柠檬口味茶饮品 500ml_瓶"
top1_candidate = "康师傅 柠檬口味冰红茶柠檬口味茶饮品 1000ml_瓶3（新老包装随机发货）"
top2_candidate = "康师傅 冰红茶柠檬味大瓶饮品 1000ml_瓶"
top3_candidate = "康师傅 茉莉清茶调味茶饮品 500ml_瓶"

In [43]:
prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)

prompt_format

'\n你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品的索引值。\n\n<Task>\n你需要按照下面步骤进行思考处理和返回：\n1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为候选匹配商品信息。\n2. 每个商品信息包含品牌、商品名、规格数量3部分信息，分析时先提取 origin_product 输入商品的品牌、商品名、规格数量信息。\n3. 提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、商品名、规格数量信息，判断是否为同款商品。\n4. 根据匹配规则选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。\n</Task>\n\n<Information Extraction Rules>\n以下为信息提取规则：\n1.商品品牌：通常位于信息前部，一般使用空格分离，部分商品可能不含品牌名。如：旺仔 精选进口乳源原味小馒头 14g_袋，品牌提取为旺仔；部分品牌和商品名重叠，如 元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，品牌为元气森林；\n2.商品名称：通常位于信息中部，需要去除描述部分。如：元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，名称提取为青柠口味电解质饮料；\n3.商品规格数量：\n    1）通常位于信息尾部，由规格和数量两部分组成。如：喜力Heineken 11.4°P啤酒 500ml3罐_包，规格为500ml3罐_包，数量为1；\n    2）部分商品数量信息位于尾部且只有数字，如：乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货），规格数量部分为500ml_听3，规格为500ml_听，数量为3；\n    3）如果没有数量部分默认为1，如：旺仔 精选进口乳源原味小馒头 14g_袋，规格为14g_袋，数量为1；\n</Information Extraction Rules>\n\n<

In [ ]:
rank_prompt_no_reason_format = rank_prompt_no_reason.format(origin_product=origin_product,
                                                            top1_candidate=top1_candidate,
                                                            top2_candidate=top2_candidate,
                                                            top3_candidate=top3_candidate)

In [46]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)

# model_qwen_8b 规格错误
# model_glm_9b 正确但不稳定
# model_glm_z1_9b 规格错误
# model_qwen3_14b 正确但不稳定
# model_qwen3_30b_thinking 指定遵循失败
# model_qwen3_30b 正确
# model_qwen3_next_80b 正确
# model_glm4_5_air 正确
# model_qwen3_235b_thinking json模式指定遵循失败
# model_longcat_flash 正确，分析准确
rankSelect = model_qwen3_30b.invoke([HumanMessage(content=prompt_format)],
                                    # config={"callbacks": [opik_tracer]}
                                    )


In [47]:
rankSelect

RankSelect(match_index=0, match_reason='首先对origin_product和各候选商品进行信息提取：\n\n1. origin_product：康师傅 冰力十足冰红茶柠檬口味茶饮品 500ml_瓶\n   - 品牌：康师傅（位于开头，明确）\n   - 商品名：冰红茶柠檬口味茶饮品（去除‘冰力十足’为营销词，保留核心品类与口味描述）\n   - 规格数量：规格为500ml_瓶，数量为1（尾部为500ml_瓶，无显式数量，默认为1）\n\n2. top1_candidate：康师傅 柠檬口味冰红茶柠檬口味茶饮品 1000ml_瓶3（新老包装随机发货）\n   - 品牌：康师傅（一致）\n   - 商品名：柠檬口味冰红茶柠檬口味茶饮品（与origin_product的‘冰红茶柠檬口味茶饮品’语义高度一致，虽多一个‘柠檬口味’重复，但核心为冰红茶+柠檬口味，可视为一致）\n   - 规格数量：规格为1000ml_瓶，数量为3（因有‘3’，数量为3）\n   → 问题：规格数量不一致，origin_product为500ml_瓶×1，top1_candidate为1000ml_瓶×3，规格和数量均不同，不符合严格匹配规则，不匹配。\n\n3. top2_candidate：康师傅 冰红茶柠檬味大瓶饮品 1000ml_瓶\n   - 品牌：康师傅（一致）\n   - 商品名：冰红茶柠檬味大瓶饮品（与origin_product的‘冰红茶柠檬口味茶饮品’相比，名称中缺少‘茶饮品’，且用‘大瓶’替代具体规格，语义不完全一致，且‘大瓶’为模糊描述，不能等同于‘500ml_瓶’）\n   - 规格数量：规格为1000ml_瓶，数量为1（无数量数字，视为1）\n   → 问题：规格数量不一致（1000ml vs 500ml），且商品名语义不完全一致（‘大瓶饮品’无法对应‘500ml_瓶’），不匹配。\n\n4. top3_candidate：康师傅 茉莉清茶调味茶饮品 500ml_瓶\n   - 品牌：康师傅（一致）\n   - 商品名：茉莉清茶调味茶饮品（与origin_product的‘冰红茶柠檬口味茶饮品’完全不同，口味与品类均不一致，明显不是同一款）\n   - 规格数量：规格为500ml_瓶，数量为1（与origin_product规格

In [20]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)

# model_qwen_8b 规格错误
# model_glm_9b 正确但不稳定
# model_glm_z1_9b 规格错误
# model_qwen3_14b 正确但不稳定
# model_qwen3_30b_thinking 指定遵循失败
# model_qwen3_30b 正确
# model_qwen3_next_80b 正确
# model_glm4_5_air 正确
# model_qwen3_235b_thinking json模式指定遵循失败
# model_longcat_flash 正确，分析准确
model_qwen3_30b_no_reason.invoke([HumanMessage(content=rank_prompt_no_reason)],
                                 config={"callbacks": [opik_tracer]})


MatchSelect(match_index=0)

In [ ]:
rank_prompt_v1 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的编号值。
你需要按照下面步骤进行思考处理和返回：
- 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
- 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
- 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
- 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
- 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. candidate信息为空，当前判断的候选商品不匹配；评判时当前候选商品信息为空可跳过，全部候选商品为空时直接返回0。
2. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒，语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
3. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1；评判时注意抑语义相同及规格一致，如1L和1000ml为一致；个、袋、盒等单个规格在整体语义下也可一致。
4. 商品名语义不符合，origin_product识别为识字卡，candidate识别为挪车卡。
5. 商品描述有冲突，origin_product识别描述冬季保暖，candidate识别描述为夏季防晒；评判时要求语义偏移或不同不视为不一致，只有完全冲突时才视为不匹配，如冬天与夏天，辣味与甜味。
6. 商品信息整体语义冲突，origin_product商品信息为小型便携式冲牙器，candidate商品信息为家用取暖器暖风机；只有比较双方商品语义完全不符合同一类商品才判断为冲突。

请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品编号. 如果是 top1_candidate 返回 1; 如果是 top2_candidate 返回 2; 如果是 top3_candidate 返回 3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
"""

In [ ]:
rank_prompt_v2 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的编号值。
你需要按照下面步骤进行思考处理和返回：
- 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
- 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
- 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
- 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
- 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒;语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1；评判时注意抑语义相同及规格一致，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下也可一致。
3. 商品名语义不一致，origin_product识别为识字卡，candidate识别为挪车卡；评断时名称不同但语义相同时视为一致，对比双方完全不是同一类型商品才会判断为不一致。
4. 商品描述完全不一致，origin_product识别描述冬季保暖羽绒服，candidate识别描述为夏季防晒衣；评判时要求语义偏移或不同不视为不一致，只有完全冲突时才视为不匹配，如冬天与夏天，辣味与甜味。

请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品编号. 如果是 top1_candidate 返回 1; 如果是 top2_candidate 返回 2; 如果是 top3_candidate 返回 3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
"""

In [ ]:
rank_prompt_v3 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
2. 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
3. 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
4. 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
5. 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Guidelines>
以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒;语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1；评判时注意抑语义相同及规格一致，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下也可一致。
3. 商品名语义不一致，origin_product识别为识字卡，candidate识别为挪车卡；评断时名称不同但语义相同时视为一致，对比双方完全不是同一类型商品才会判断为不一致。
4. 商品描述完全不一致，origin_product识别描述冬季保暖羽绒服，candidate识别描述为夏季防晒衣；评判时要求语义偏移或不同不视为不一致，只有完全冲突时才视为不匹配，如冬天与夏天，辣味与甜味。
</Guidelines>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品索引. 如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>
"""

In [ ]:
rank_prompt_v4 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品类型的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
2. 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
3. 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
4. 依次提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、描述、商品名、规格数量信息，判断是否为同款商品。
5. 全面分析思考，选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

<Guidelines>
以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品为不同商品：
1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时由于提取品牌可能包含商品信息，因此商品部分词汇重叠也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，规格数量需要同时满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位；评判时注意规格单位换算，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下视为同等规格。
3. 商品名语义或分类不一致，origin_product识别为识字卡，candidate识别为挪车卡。
4. 商品同类型描述完全不一致，origin_product提取描述为橘子味，candidate提取描述为葡萄味。
</Guidelines>

<Examples>
示例1：
origin_product：卫龙魔芋爽 微辣麻酱素毛肚 （15+3）g_袋
top1_candidate：卫龙 魔芋爽 香辣味素毛肚 15克_袋
分析：origin_product和top1_candidate品牌、商品名都为卫龙和魔芋爽，但是由于规格数量不同，origin_product为18g/袋,top1_candidate为15g/袋，所以不是同款商品。

示例2：
origin_product：【毛绒长款睡袍】卡皮巴拉冬季新款加厚睡袍卡通可爱风珊瑚绒睡衣女家居服_件
top1_candidate：珊瑚绒睡袍女冬季甜美加厚新款长款睡裙法兰绒睡衣女可外穿家居服_件（没有裤子）
分析：origin_product和top1_candidate品牌、商品无法提取，商品描述未高度重叠，所有判断不是同款商品，需要返回0，match_index=0。
</Examples>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回origin_product描述一致的候选商品索引。 如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；没有候选商品匹配或其他情况返回0。
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>

<Critical Reminder>
- 规格数量未必须条件，需满足完全一致，基本一致视为不匹配。
- 当商品提取不到品牌、数量规格信息时，商品名称和商品描述需要高度一致才视为同一款商品。
- 特别注意，如果候选商品信息都不是同款，不需要考虑寻找最接近的候选商品，说明没有同款商品，直接返回0。
- 必须保证分析结果编号或索引与返回的match_index字段值相同，分析得出没有匹配结果时返回0。
</<Critical Reminder>
"""

In [ ]:

rank_prompt_v5 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品类型的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为候选匹配商品信息。
2. 每个商品信息包含品牌、描述、商品名、规格数量4部分信息，分析时先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
3. 依次提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、描述、商品名、规格数量信息，判断是否为同款商品。
4. 全面分析思考，根据匹配规则选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

<Information Extraction Rules>
以下为信息提取规则：
1.商品品牌：通常位于信息前部，使用空格分离，部分商品可能不含品牌名。如：旺仔 精选进口乳源原味小馒头 14g_袋，品牌提取为旺仔；部分品牌和商品名重叠，如 元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，品牌为元气森林；
2.商品描述：同上位于信息中部。如：统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶，商品描述为 青春无极限、柠檬味茶饮料；
3.商品名称：通常位于信息中部。如：统一企业茄皇 茄皇鸡蛋面 120g_桶，名称提取为茄皇鸡蛋面；
4.商品规格数量：1）通常位于信息尾部，由规格和数量两部分组成。如：喜力Heineken 11.4°P啤酒 500ml3罐_包，规格为500ml，数量为3罐_包；2）部分数量信息位于尾部只有数字，如：乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货），规格数量部分为500ml_听3，规格为500ml_听，数量为3；3）如果没有数量部分默认为1，如：旺仔 精选进口乳源原味小馒头 14g_袋，规格为14g_袋，数量为1；
</Information Extraction Rules>

<Match rules>
以下是origin_product和候选商品的匹配规则：
1. 商品品牌不一致时视为不匹配，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时由于提取品牌可能包含商品信息，因此商品部分词汇重叠也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 如果origin_product和top_candidate任意一方没有品牌时视为不匹配.
3. 商品数量和规格都一致才视为匹配，数据和规格不一致标明不是同款商品，匹配度为0。规格可包括L、kg、千克、盒、件、片、条等市面计量单位；评判时注意规格单位换算，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下视为同等规格；如 1000ml_瓶和1000ml_瓶2数量不匹配。
4. 商品名语义或分类不一时致视为不匹配，origin_product识别为识字卡，candidate识别为挪车卡。
5. 商品同类型描述完全不一致时视为不匹配，origin_product提取描述为橘子味，candidate提取描述为葡萄味。
6. 候选top_candidate都不匹配时不需要寻找最接近的商品，视为没有商品匹配，返回0。
</Match rules>

<Examples>
示例1：
origin_product：卫龙魔芋爽 微辣麻酱素毛肚 （15+3）g_袋
top1_candidate：卫龙 魔芋爽 香辣味素毛肚 15克_袋
分析：origin_product和top1_candidate品牌、商品名都为卫龙和魔芋爽，但是由于规格数量不同，origin_product为18g/袋,top1_candidate为15g/袋，所以不是同款商品。

示例2：
origin_product：【毛绒长款睡袍】卡皮巴拉冬季新款加厚睡袍卡通可爱风珊瑚绒睡衣女家居服_件
top1_candidate：珊瑚绒睡袍女冬季甜美加厚新款长款睡裙法兰绒睡衣女可外穿家居服_件（没有裤子）
分析：origin_product和top1_candidate品牌、商品无法提取，商品描述未高度重叠，所有判断不是同款商品，需要返回0，match_index=0。
</Examples>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回origin_product描述一致的候选商品索引。 如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；没有候选商品匹配或其他情况返回0。
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>

<Critical Reminder>
1. 规格数量未必须条件，需满足完全一致，基本一致视为不匹配。
2. 当商品提取不到品牌、数量规格信息时，商品名称和商品描述需要高度一致才视为同一款商品。
3. 特别注意，如果候选商品信息都不是同款，不需要考虑寻找最接近的候选商品，说明没有同款商品，直接返回0。
4. 必须保证分析结果编号或索引与返回的match_index字段值相同，分析得出没有匹配结果时返回0。
</<Critical Reminder>
"""

In [ ]:

rank_prompt_v6 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品类型的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为候选匹配商品信息。
2. 每个商品信息包含品牌、商品名、规格数量4部分信息，分析时先提取 origin_product 输入商品的品牌、商品名、规格数量信息。
3. 依次提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、商品名、规格数量信息，判断是否为同款商品。
4. 全面分析思考，根据匹配规则选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

<Information Extraction Rules>
以下为信息提取规则：
1.商品品牌：通常位于信息前部，使用空格分离，部分商品可能不含品牌名。如：旺仔 精选进口乳源原味小馒头 14g_袋，品牌提取为旺仔；部分品牌和商品名重叠，如 元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，品牌为元气森林；
2.商品名称：通常位于信息中部。如：元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，名称提取为青柠口味电解质饮料；
3.商品规格数量：1）通常位于信息尾部，由规格和数量两部分组成。如：喜力Heineken 11.4°P啤酒 500ml3罐_包，规格为500ml，数量为3罐_包；2）部分数量信息位于尾部只有数字，如：乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货），规格数量部分为500ml_听3，规格为500ml_听，数量为3；3）如果没有数量部分默认为1，如：旺仔 精选进口乳源原味小馒头 14g_袋，规格为14g_袋，数量为1；
</Information Extraction Rules>

<Match rules>
以下是origin_product和候选商品的匹配规则：
1. 商品品牌不一致时视为不匹配，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时由于提取品牌可能包含商品信息，因此商品部分词汇重叠也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 如果origin_product和top_candidate任意一方没有品牌时视为不匹配.
3. 商品数量和规格都一致才视为匹配，数据和规格不一致标明不是同款商品，匹配度为0。1）规格可包括L、kg、千克、盒、件、片、条等市面计量单位；2）评判时注意规格单位换算，如1L和1000ml为一致；3）个、袋、盒、瓶等单个包装规格在整体语义下视为同等规格；4）数量不一致如： 1000ml_瓶和1000ml_瓶2数量不匹配；5）规格中没有体现数量，视为数量为1。
4. 商品名语义或分类不一时致视为不匹配，origin_product识别为识字卡，candidate识别为挪车卡。
5. 候选top_candidate都不匹配时不需要寻找最接近的商品，视为没有商品匹配，返回0。
</Match rules>

<Examples>
示例1：
origin_product：卫龙魔芋爽 微辣麻酱素毛肚 （15+3）g_袋
top1_candidate：卫龙 魔芋爽 香辣味素毛肚 15克_袋
分析：origin_product和top1_candidate品牌、商品名都为卫龙和魔芋爽，但是由于规格数量不同，origin_product为18g/袋,top1_candidate为15g/袋，所以不是同款商品。

示例2：
origin_product：【毛绒长款睡袍】卡皮巴拉冬季新款加厚睡袍卡通可爱风珊瑚绒睡衣女家居服_件
top1_candidate：珊瑚绒睡袍女冬季甜美加厚新款长款睡裙法兰绒睡衣女可外穿家居服_件（没有裤子）
分析：origin_product和top1_candidate品牌、商品无法提取，商品描述未高度重叠，所有判断不是同款商品，需要返回0，match_index=0。
</Examples>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回origin_product描述一致的候选商品索引；没有候选商品匹配或其他情况返回0。
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>

<Critical Reminder>
1. 规格数量未必须条件，需满足完全一致，基本一致视为不匹配。
2. 当商品提取不到品牌、数量规格信息时，商品名称和商品描述需要高度一致才视为同一款商品。
3. 特别注意，如果候选商品信息都不是同款，不需要考虑寻找最接近的候选商品，说明没有同款商品，直接返回0。
4. 必须保证分析结果编号或索引与返回的match_index字段值相同，分析得出没有匹配结果时返回0。
</<Critical Reminder>
"""

In [ ]:
# v2
rank_prompt_v7 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品中选择描述为同一款商品的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为候选匹配商品信息。
2. 每个商品信息包含品牌、商品名、规格数量3部分信息，分析时先提取 origin_product 输入商品的品牌、商品名、规格数量信息。
3. 提取并对比origin_product与候选商品top1_candidate,top2_candidate,top3_candidate的品牌、商品名、规格数量信息，判断是否为同款商品。
4. 根据匹配规则选出与origin_product描述为同款商品的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

<Information Extraction Rules>
以下为信息提取规则：
1.商品品牌：通常位于信息前部，一般使用空格分离，部分商品可能不含品牌名。如：旺仔 精选进口乳源原味小馒头 14g_袋，品牌提取为旺仔；部分品牌和商品名重叠，如 元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，品牌为元气森林；
2.商品名称：通常位于信息中部，需要去除描述部分。如：元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶，名称提取为青柠口味电解质饮料；
3.商品规格数量：
    1）通常位于信息尾部，由规格和数量两部分组成。如：喜力Heineken 11.4°P啤酒 500ml3罐_包，规格为500ml3罐_包，数量为1；
    2）部分商品数量信息位于尾部且只有数字，如：乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货），规格数量部分为500ml_听3，规格为500ml_听，数量为3；
    3）如果没有数量部分默认为1，如：旺仔 精选进口乳源原味小馒头 14g_袋，规格为14g_袋，数量为1；
</Information Extraction Rules>

<Match rules>
以下是origin_product和候选商品的匹配规则：
1. 商品品牌不一致时视为不匹配，例如origin_product识别品牌为可口，candidate识别为品牌为百事；特别注意，评判时由于提取品牌可能包含商品信息，因此商品部分词汇重叠也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 如果origin_product和top_candidate任意一方没有品牌时视为不匹配.
3. 商品数量和规格严格一致才视为匹配，数据和规格不一致标明不是同款商品，匹配度为0。以下是规格数据匹配时处理注意事项：
    1）规格可包括L、kg、千克、盒、件、片、条等市面计量单位，评判时注意规格单位换算，如1L和1000ml为一致；
    2）个、袋、盒、瓶等单个包装规格在整体语义下视为同等规格；
    3）规格中没有体现数量，视为数量为1。
4. 商品名称语义不一致视为不匹配，origin_product识别为识字卡，candidate识别为挪车卡。
5. 候选top_candidate都不匹配时不需要寻找最接近的商品，视为没有商品匹配，返回0。
</Match rules>

<Examples>
示例1：
origin_product：卫龙魔芋爽 微辣麻酱素毛肚 （15+3）g_袋
top1_candidate：卫龙 魔芋爽 香辣味素毛肚 15克_袋
分析：origin_product和top1_candidate品牌、商品名都为卫龙和魔芋爽，但是由于规格数量不同，origin_product为18g/袋,top1_candidate为15g/袋，所以不是同款商品。

示例2：
origin_product：【毛绒长款睡袍】卡皮巴拉冬季新款加厚睡袍卡通可爱风珊瑚绒睡衣女家居服_件
top1_candidate：珊瑚绒睡袍女冬季甜美加厚新款长款睡裙法兰绒睡衣女可外穿家居服_件（没有裤子）
分析：origin_product和top1_candidate品牌、商品无法提取，商品描述未高度重叠，所有判断不是同款商品，需要返回0，match_index=0。
</Examples>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回origin_product描述一致的候选商品索引；没有候选商品匹配或其他情况返回0。
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>

<Critical Reminder>
1. 规格数量严格匹配为必须条件，基本一致视为不匹配。
2. 当商品提取不到品牌、数量规格信息时，商品名称和商品描述需要高度一致才视为同一款商品。
3. 特别注意，如果候选商品信息都不是同款，不需要考虑寻找最接近的候选商品，说明没有同款商品，直接返回0。
4. 必须保证分析结果编号或索引与返回的match_index字段值相同，分析得出没有匹配结果时返回0。
</Critical Reminder>
"""

In [8]:
# 对下面格式字段封装列表，进行批量化调用
# origin_product = "迪味臻 五香卤蛋 30g_袋"
# top1_candidate = "迪味臻 五香卤蛋 30g_个"
# top2_candidate = ""
# top3_candidate = ""
# 1.规格问题，很多名称中带有瓶2、瓶3，单位后面带数字的这种，大模型都会判断错；
# 2.大模型匹配分析index与输出index不一致问题，大模型输出稳定性的问题，可能要换更大模型；
# 3.召回问题，条码匹配规则会影响召回可以考虑去掉，然后可以试试放宽到Top5；
# 4.召回问题，一些记录给出的相似商品不满3个，而且没召回手工打标的那个，按道理不应该；
# 5.品牌问题，之前的规则是两边都有品牌且不一致则认为不一致，但是看了下样本中，没有品牌信息的商品大部分人工打标都是空的，也就是说没有品牌的话，是相同商品的概率比较小，可以考虑把规则改成，如果任意一方无品牌，就判为不一致；
# 6.属性判断不准问题，类似衣服款式（如开衫）、口味（如香辣味）这种，大模型会比较宽松，但是如果变严格，可能又会把一些原本判断对的误杀了，不太好把握。
# 定义示例数据列表
examples = []

index_error_1 = {"original_product": "乌苏 11°P乌苏啤酒罐装 500ml_听",
                 "top1_candidate": "乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）",
                 "top2_candidate": "雪花啤酒 原汁麦8°P啤酒罐装 500ml_听",
                 "top3_candidate": "乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐",
                 "cr": "乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐"}
index_error_2 = {"original_product": "元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶",
                 "top1_candidate": "元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶",
                 "top2_candidate": "元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶",
                 "top3_candidate": "气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）",
                 "cr": "元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶"}
index_error_3 = {"original_product": "统一 老坛酸菜牛肉面 120g_桶",
                 "top1_candidate": "统一  老坛酸菜牛肉面  120g_桶2",
                 "top2_candidate": "【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶",
                 "top3_candidate": "统一企业茄皇 茄皇鸡蛋面 120g_桶",
                 "cr": "【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶"}

no_match_error_1 = {"original_product": "旺仔 精选进口乳源原味小馒头 14g_袋",
                    "top1_candidate": "旺仔 小馒头 独立小包散装牛奶原味14g_袋",
                    "top2_candidate": "旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋",
                    "top3_candidate": "",
                    "cr": ""}
no_match_error_2 = {
    "original_product": "【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖",
    "top1_candidate": "【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条",
    "top2_candidate": "【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条",
    "top3_candidate": "【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条",
    "cr": ""}
no_match_error_3 = {
    "original_product": "【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服",
    "top1_candidate": "珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套",
    "top2_candidate": "【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套",
    "top3_candidate": "睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套",
    "cr": ""}
num_error_1 = {"original_product": "统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶",
               "top1_candidate": "统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2",
               "top2_candidate": "统一 冰红茶柠檬味茶饮料 1l4瓶",
               "top3_candidate": "一冰红茶 柠檬味冰红茶 1L_瓶",
               "cr": "统一冰红茶 柠檬味冰红茶 1L_瓶"}
num_error_2 = {"original_product": "元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶",
               "top1_candidate": "元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶'",
               "top2_candidate": "元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）",
               "top3_candidate": "",
               "cr": "元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶"}
num_error_3 = {"original_product": "百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋",
               "top1_candidate": "百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2'",
               "top2_candidate": "乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",
               "top3_candidate": "乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋",
               "cr": "乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋"}
similar_error_1 = {"original_product": "喜力Heineken 11.4°P啤酒 500ml3罐_包",
                   "top1_candidate": "喜力Heineken 11.4°P啤酒 500ml3罐_组",
                   "top2_candidate": "喜力Heineken 11.4°P啤酒 500ml6罐_箱",
                   "top3_candidate": "喜力Heineken 11.4°P罐装啤酒 500ml3听罐",
                   "cr": "喜力Heineken 11.4°P罐装啤酒 500ml3听罐"}
similar_error_2 = {"original_product": "力士幽莲魅肤精油香氛凝萃恒香精油沐浴露720克_瓶",
                   "top1_candidate": "力士 沐浴露幽莲魅肤大容量家庭装香氛香薰男女通用持久留香720ml_瓶",
                   "top2_candidate": "力士 沐浴露 恒久嫩肤淡雅香氛 精油沐浴露持久留香 720g_瓶",
                   "top3_candidate": "【沐浴露】梵贞氨基酸精油香氛润肤沐浴露800ml_瓶",
                   "cr": "力士 沐浴露幽莲魅肤大容量家庭装香氛香薰男女通用持久留香720ml_瓶"}
similar_error_3 = {"original_product": "【无痕强力】透明挂钩1个 壁挂式粘钩墙钩家用瓷砖粘墙钩免打孔杂物挂钩无痕挂钩~",
                   "top1_candidate": "挂钩强力粘胶墙壁粘免打孔壁挂式承重粘钩挂勾无痕粘墙粘贴粘钩1个颜色随机钩",
                   "top2_candidate": "透明挂钩 卫生间免打孔粘钩置物架专用门后透明无痕贴厨房浴室无痕钉无痕钩1个",
                   "top3_candidate": "【尺寸可选】免打孔挂钉挂钩 强力墙钉隐形钉相框照片墙无痕钉子相框免打孔挂钩 1个",
                   "cr": "透明挂钩 卫生间免打孔粘钩置物架专用门后透明无痕贴厨房浴室无痕钉无痕钩1个"}
loss_error_1 = {"original_product": "【无痕裤夹】干湿两用黑色294.1cm_1个 裤子裤架家用防滑裤架挂衣裙夹塑料衣撑子~",
                "top1_candidate": "日系无痕裤架JK群子夹家用可连挂裤夹子裙夹西裤衣柜收纳防皱 衣架1个_颜色随机",
                "top2_candidate": "【32cm】家用防风木头带夹子裤架  实用伸缩金属衣架 衣柜内夹裤子衣架 1个",
                "top3_candidate": "",
                "cr": "日系无痕裤架JK群子夹家用可连挂裤夹子裙夹西裤衣柜收纳防皱 衣架1个_颜色随机"}
loss_error_2 = {"original_product": "硅胶地漏 卫生间洗脸盆浴室厨房下水道防虫防臭地漏芯加厚整套 蓝色 一个",
                "top1_candidate": "'硅胶防臭地漏芯卫生间下水道排水口洗手池防止反味神器洗防臭味盖 1个",
                "top2_candidate": "地漏防臭器下水道管堵口厕所通用卫生间除防返臭反味神器防虫盖芯1个【颜色随机】",
                "top3_candidate": "",
                "cr": "硅胶防臭地漏芯卫生间下水道排水口洗手池防止反味神器洗防臭味盖 1个"}
loss_error_3 = {"original_product": "可心柔婴儿柔纸巾云柔巾宝宝家用乳霜纸宝宝保湿纸巾40抽_包X",
                "top1_candidate": "可心柔婴儿柔纸巾3层40抽 120张_包 新生儿纸巾便携装婴儿抽纸",
                "top2_candidate": "【大包带盖】天真贝比 婴儿湿巾纸宝宝手口湿纸巾新生儿童湿巾50片_包",
                "top3_candidate": "",
                "cr": "可心柔婴儿柔纸巾3层40抽 120张_包 新生儿纸巾便携装婴儿抽纸"}
examples.append(index_error_1)
examples.append(index_error_2)
examples.append(index_error_3)
examples.append(no_match_error_1)
examples.append(no_match_error_2)
examples.append(no_match_error_3)
examples.append(num_error_1)
examples.append(num_error_2)
examples.append(num_error_3)
examples.append(similar_error_1)
examples.append(similar_error_2)
examples.append(similar_error_3)
examples.append(loss_error_1)
examples.append(loss_error_2)
examples.append(loss_error_3)


In [25]:
# 迭代调用模型，存储结果到列表
# model_qwen3_next_80b
# model_qwen3_30b
# model_qwen3_max
# model_qwen_8b
# model_glm_9b
# model_qwen3_235b_instruct
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)
result_list_model_qwen_8b = []
for example in examples:
    # 获取元素
    origin_product = example.get("original_product")
    top1_candidate = example.get("top1_candidate")  
    top2_candidate = example.get("top2_candidate")
    top3_candidate = example.get("top3_candidate")
    prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
    rankSelect = model_qwen_8b.invoke([HumanMessage(content=prompt_format)],
                                    config={"callbacks": [opik_tracer]})
    result_list_model_qwen_8b.append(rankSelect)

In [11]:
result_list_model_qwen_8b

[RankSelect(match_index=1, match_reason="首先提取origin_product的商品信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒罐装'，规格数量为'500ml_听'。接着分析top1_candidate信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒'，规格数量为'500ml_听3（新老包装随机发货）'。分析可知，品牌一致；商品名称部分重叠，均指的是'乌苏啤酒'，存在语义一致性；规格数量部分虽有数量差异，但都以'500ml_听'为基础，且top1_candidate的数量3是附属说明，不影响'规格'部分的识别。因此，top1_candidate与origin_product可视为同款商品。top2_candidate品牌不一致，top3_candidate商品名称和描述与origin_product存在差异，故不匹配。"),
 RankSelect(match_index=1, match_reason='首先提取origin_product的信息：品牌为元气森林，商品名为外星人电解质水，描述为0糖0卡青柠口味电解质饮料，规格数量为500ml_瓶。然后依次对比候选商品top1_candidate、top2_candidate、top3_candidate的信息。'),
 RankSelect(match_index=1, match_reason='首先，提取 origin_product 的信息：品牌为统一，商品名为老坛酸菜牛肉面，规格数量为120g_桶。接下来分别分析候选商品：top1_candidate 的品牌为统一，商品名为老坛酸菜牛肉面，规格数量为120g_桶2；top2_candidate 的品牌无法提取（因为信息前部有【单桶】说明，可能属于包装属性），商品名称是统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面，规格数量为120g_桶，但品牌缺失，因此不符合匹配规则；top3_candidate 的品牌为统一企业，商品名为茄皇鸡蛋面，规格数量为120g_桶。因此，只有 top1_candidate 与 origin_product 在品牌、商品名及规格数量上完全一致，可以判定为同款商品。其他候选商品由于品牌或商品名不一致无法匹配。'),
 RankSelect(match_index=1, match_reason

In [28]:
import pandas as pd

df = pd.DataFrame(obj for obj in examples)
df

,original_product,top1_candidate,top2_candidate,top3_candidate,cr
0,乌苏 11°P乌苏啤酒罐装 500ml_听,乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）,雪花啤酒 原汁麦8°P啤酒罐装 500ml_听,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐
1,元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶,气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶
2,统一 老坛酸菜牛肉面 120g_桶,统一 老坛酸菜牛肉面 120g_桶2,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,统一企业茄皇 茄皇鸡蛋面 120g_桶,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶
3,旺仔 精选进口乳源原味小馒头 14g_袋,旺仔 小馒头 独立小包散装牛奶原味14g_袋,旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋,,
4,【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖,【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条,【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条,【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条,
5,【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服,珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套,【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套,睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套,
6,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2,统一 冰红茶柠檬味茶饮料 1l4瓶,一冰红茶 柠檬味冰红茶 1L_瓶,统一冰红茶 柠檬味冰红茶 1L_瓶
7,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶',元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）,,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶
8,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2',"乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋
9,喜力Heineken 11.4°P啤酒 500ml3罐_包,喜力Heineken 11.4°P啤酒 500ml3罐_组,喜力Heineken 11.4°P啤酒 500ml6罐_箱,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,喜力Heineken 11.4°P罐装啤酒 500ml3听罐


In [29]:
# 转为pandas 对象,列名指定

import pandas as pd

result_df_model_qwen_8b = pd.DataFrame(obj.dict() for obj in result_list_model_qwen_8b)
result_df_model_qwen_8b.rename(columns={"match_index": "match_index_model_qwen_8b"}, inplace=True)
result_df_model_qwen_8b.rename(columns={"match_reason": "match_reason_model_qwen_8b"}, inplace=True)
df = pd.concat([df.reset_index(drop=True),
                    result_df_model_qwen_8b.reset_index(drop=True)], axis=1)
df

/var/folders/wt/qx53z0x950jdy1qsklrdghq40000gn/T/ipykernel_78422/3566963949.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_df_model_qwen_8b = pd.DataFrame(obj.dict() for obj in result_list_model_qwen_8b)


,original_product,top1_candidate,top2_candidate,top3_candidate,cr,match_index_model_qwen_8b,match_reason_model_qwen_8b
0,乌苏 11°P乌苏啤酒罐装 500ml_听,乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）,雪花啤酒 原汁麦8°P啤酒罐装 500ml_听,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,1,首先提取origin_product的商品信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒罐...
1,元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶,气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,1,首先提取origin_product的信息：品牌为元气森林，商品名为外星人电解质水，描述为0...
2,统一 老坛酸菜牛肉面 120g_桶,统一 老坛酸菜牛肉面 120g_桶2,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,统一企业茄皇 茄皇鸡蛋面 120g_桶,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,1,首先，提取 origin_product 的信息：品牌为统一，商品名为老坛酸菜牛肉面，规格数...
3,旺仔 精选进口乳源原味小馒头 14g_袋,旺仔 小馒头 独立小包散装牛奶原味14g_袋,旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋,,,1,首先提取origin_product的信息：品牌为旺仔，商品名为精选进口乳源原味小馒头，规格...
4,【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖,【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条,【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条,【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条,,1,首先提取origin_product的信息：品牌为'无'，商品名为'简约经典千鸟格围巾'，描...
5,【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服,珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套,【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套,睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套,,3,首先对origin_product进行信息提取：品牌部分无法识别，商品名称为'珊瑚绒睡衣'，...
6,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2,统一 冰红茶柠檬味茶饮料 1l4瓶,一冰红茶 柠檬味冰红茶 1L_瓶,统一冰红茶 柠檬味冰红茶 1L_瓶,1,首先，提取商品信息：origin_product的品牌是'统一'，商品名称是'青春无极限冰红...
7,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶',元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）,,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,1,首先提取origin_product的信息：品牌为元气，商品名为自在水，商品描述为0糖0脂红...
8,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2',"乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,1,首先提取origin_product的信息：品牌为‘百事食品乐事’，商品名为‘意大利香浓红烩...
9,喜力Heineken 11.4°P啤酒 500ml3罐_包,喜力Heineken 11.4°P啤酒 500ml3罐_组,喜力Heineken 11.4°P啤酒 500ml6罐_箱,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,1,首先，提取origin_product的信息：品牌为喜力Heineken，商品名为11.4°...


In [30]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)
# 迭代调用模型，存储结果到列表
# model_qwen3_next_80b
# model_qwen3_30b
# model_qwen3_max
# model_qwen_8b
# model_glm_9b
# model_qwen3_235b_instruct
result_list_model_qwen3_30b = []
for example in examples:
    # 获取元素
    origin_product = example.get("original_product")
    top1_candidate = example.get("top1_candidate")  
    top2_candidate = example.get("top2_candidate")
    top3_candidate = example.get("top3_candidate")
    prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
    rankSelect = model_qwen3_30b.invoke([HumanMessage(content=prompt_format)],
                                    config={"callbacks": [opik_tracer]})
    result_list_model_qwen3_30b.append(rankSelect)
    
import pandas as pd

result_df_model_qwen3_30b = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_30b)
result_df_model_qwen3_30b.rename(columns={"match_index": "match_index_model_qwen3_30b"}, inplace=True)
result_df_model_qwen3_30b.rename(columns={"match_reason": "match_reason_model_qwen3_30b"}, inplace=True)
df = pd.concat([df.reset_index(drop=True),
                    result_df_model_qwen3_30b.reset_index(drop=True)], axis=1)
df

/var/folders/wt/qx53z0x950jdy1qsklrdghq40000gn/T/ipykernel_78422/1487263109.py:27: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_df_model_qwen3_30b = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_30b)


,original_product,top1_candidate,top2_candidate,top3_candidate,cr,match_index_model_qwen_8b,match_reason_model_qwen_8b,match_index_model_qwen3_30b,match_reason_model_qwen3_30b
0,乌苏 11°P乌苏啤酒罐装 500ml_听,乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）,雪花啤酒 原汁麦8°P啤酒罐装 500ml_听,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,1,首先提取origin_product的商品信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒罐...,1,首先提取origin_product的信息：品牌为'乌苏'，描述为'11°P乌苏啤酒罐装'，...
1,元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶,气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,1,首先提取origin_product的信息：品牌为元气森林，商品名为外星人电解质水，描述为0...,1,首先提取origin_product信息：品牌为元气森林，商品描述为0糖0卡青柠口味电解质饮...
2,统一 老坛酸菜牛肉面 120g_桶,统一 老坛酸菜牛肉面 120g_桶2,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,统一企业茄皇 茄皇鸡蛋面 120g_桶,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,1,首先，提取 origin_product 的信息：品牌为统一，商品名为老坛酸菜牛肉面，规格数...,1,首先提取origin_product的信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格...
3,旺仔 精选进口乳源原味小馒头 14g_袋,旺仔 小馒头 独立小包散装牛奶原味14g_袋,旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋,,,1,首先提取origin_product的信息：品牌为旺仔，商品名为精选进口乳源原味小馒头，规格...,1,首先对origin_product进行信息提取：品牌为'旺仔'，商品描述为'精选进口乳源原味...
4,【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖,【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条,【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条,【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条,,1,首先提取origin_product的信息：品牌为'无'，商品名为'简约经典千鸟格围巾'，描...,0,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...
5,【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服,珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套,【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套,睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套,,3,首先对origin_product进行信息提取：品牌部分无法识别，商品名称为'珊瑚绒睡衣'，...,1,首先提取origin_product的信息：品牌未明确，商品描述为‘珊瑚绒睡衣’、‘秋冬开衫...
6,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2,统一 冰红茶柠檬味茶饮料 1l4瓶,一冰红茶 柠檬味冰红茶 1L_瓶,统一冰红茶 柠檬味冰红茶 1L_瓶,1,首先，提取商品信息：origin_product的品牌是'统一'，商品名称是'青春无极限冰红...,1,首先提取origin_product的信息：品牌为'统一'，商品描述为'青春无极限冰红茶柠檬...
7,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶',元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）,,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,1,首先提取origin_product的信息：品牌为元气，商品名为自在水，商品描述为0糖0脂红...,2,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...
8,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2',"乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,1,首先提取origin_product的信息：品牌为‘百事食品乐事’，商品名为‘意大利香浓红烩...,3,首先提取origin_product的信息：品牌为'百事食品乐事'（根据规则1，品牌位于信息...
9,喜力Heineken 11.4°P啤酒 500ml3罐_包,喜力Heineken 11.4°P啤酒 500ml3罐_组,喜力Heineken 11.4°P啤酒 500ml6罐_箱,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,1,首先，提取origin_product的信息：品牌为喜力Heineken，商品名为11.4°...,1,首先提取origin_product的信息：品牌为喜力Heineken，商品描述为11.4°...


In [31]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)
# 迭代调用模型，存储结果到列表
# model_qwen3_next_80b
# model_qwen3_30b
# model_qwen3_max
# model_qwen_8b
# model_glm_9b
# model_qwen3_235b_instruct
result_list_model_qwen3_next_80b = []
for example in examples:
    # 获取元素
    origin_product = example.get("original_product")
    top1_candidate = example.get("top1_candidate")  
    top2_candidate = example.get("top2_candidate")
    top3_candidate = example.get("top3_candidate")
    prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
    rankSelect = model_qwen3_next_80b.invoke([HumanMessage(content=prompt_format)],
                                    config={"callbacks": [opik_tracer]})
    result_list_model_qwen3_next_80b.append(rankSelect)
    
import pandas as pd

result_df_model_qwen3_next_80b = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_next_80b)
result_df_model_qwen3_next_80b.rename(columns={"match_index": "match_index_model_qwen3_next_80b"}, inplace=True)
result_df_model_qwen3_next_80b.rename(columns={"match_reason": "match_reason_model_qwen3_next_80b"}, inplace=True)
df = pd.concat([df.reset_index(drop=True),
                    result_df_model_qwen3_next_80b.reset_index(drop=True)], axis=1)
df

/var/folders/wt/qx53z0x950jdy1qsklrdghq40000gn/T/ipykernel_78422/1704740908.py:27: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_df_model_qwen3_next_80b = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_next_80b)


,original_product,top1_candidate,top2_candidate,top3_candidate,cr,match_index_model_qwen_8b,match_reason_model_qwen_8b,match_index_model_qwen3_30b,match_reason_model_qwen3_30b,match_index_model_qwen3_next_80b,match_reason_model_qwen3_next_80b
0,乌苏 11°P乌苏啤酒罐装 500ml_听,乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）,雪花啤酒 原汁麦8°P啤酒罐装 500ml_听,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,1,首先提取origin_product的商品信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒罐...,1,首先提取origin_product的信息：品牌为'乌苏'，描述为'11°P乌苏啤酒罐装'，...,1,首先提取origin_product信息：品牌为'乌苏'，商品描述为'11°P乌苏啤酒罐装'...
1,元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶,气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,1,首先提取origin_product的信息：品牌为元气森林，商品名为外星人电解质水，描述为0...,1,首先提取origin_product信息：品牌为元气森林，商品描述为0糖0卡青柠口味电解质饮...,1,首先提取origin_product的信息：品牌为'元气森林'，描述为'0糖0卡青柠口味电解...
2,统一 老坛酸菜牛肉面 120g_桶,统一 老坛酸菜牛肉面 120g_桶2,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,统一企业茄皇 茄皇鸡蛋面 120g_桶,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,1,首先，提取 origin_product 的信息：品牌为统一，商品名为老坛酸菜牛肉面，规格数...,1,首先提取origin_product的信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格...,1,首先提取origin_product信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，商品描...
3,旺仔 精选进口乳源原味小馒头 14g_袋,旺仔 小馒头 独立小包散装牛奶原味14g_袋,旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋,,,1,首先提取origin_product的信息：品牌为旺仔，商品名为精选进口乳源原味小馒头，规格...,1,首先对origin_product进行信息提取：品牌为'旺仔'，商品描述为'精选进口乳源原味...,1,首先，提取origin_product信息：品牌为'旺仔'，商品名为'小馒头'，商品描述为'...
4,【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖,【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条,【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条,【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条,,1,首先提取origin_product的信息：品牌为'无'，商品名为'简约经典千鸟格围巾'，描...,0,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...,1,首先，提取origin_product的信息：品牌无法提取（无明确品牌名），商品名为'千鸟格...
5,【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服,珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套,【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套,睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套,,3,首先对origin_product进行信息提取：品牌部分无法识别，商品名称为'珊瑚绒睡衣'，...,1,首先提取origin_product的信息：品牌未明确，商品描述为‘珊瑚绒睡衣’、‘秋冬开衫...,3,首先，提取origin_product信息：品牌未明确提及，商品名为'珊瑚绒睡衣'，描述为'...
6,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2,统一 冰红茶柠檬味茶饮料 1l4瓶,一冰红茶 柠檬味冰红茶 1L_瓶,统一冰红茶 柠檬味冰红茶 1L_瓶,1,首先，提取商品信息：origin_product的品牌是'统一'，商品名称是'青春无极限冰红...,1,首先提取origin_product的信息：品牌为'统一'，商品描述为'青春无极限冰红茶柠檬...,1,首先提取origin_product信息：品牌为'统一'，商品描述为'青春无极限、柠檬味茶饮...
7,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶',元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）,,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,1,首先提取origin_product的信息：品牌为元气，商品名为自在水，商品描述为0糖0脂红...,2,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...,2,首先提取origin_product信息：品牌为'元气自在水'（根据规则，品牌位于前部，'元...
8,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2',"乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,1,首先提取origin_product的信息：品牌为‘百事食品乐事’，商品名为‘意大利香浓红烩...,3,首先提取origin_product的信息：品牌为'百事食品乐事'（根据规则1，品牌位于信息...,1,首先提取origin_product信息：品牌为'百事食品乐事'，描述为'意大利香浓红烩味原...
9,喜力Heineken 11.4°P啤酒 500ml3罐_包,喜力Heineken 11.4°P啤酒 500ml3罐_组,喜力Heineken 11.4°P啤酒 500ml6罐_箱,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,1,首先，提取origin_product的信息：品牌为喜力Heineken，商品名为11.4°...,1,首先提取origin_product的信息：品牌为喜力Heineken，商品描述为11.4°...,1,首先提取origin_product信息：品牌为'喜力Heineken'，商品描述为'11....


In [32]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)
# 迭代调用模型，存储结果到列表
# model_qwen3_next_80b
# model_qwen3_30b
# model_qwen3_max
# model_qwen_8b
# model_glm_9b
# model_qwen3_235b_instruct
result_list_model_qwen3_235b_instruct = []
for example in examples:
    # 获取元素
    origin_product = example.get("original_product")
    top1_candidate = example.get("top1_candidate")  
    top2_candidate = example.get("top2_candidate")
    top3_candidate = example.get("top3_candidate")
    prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
    rankSelect = model_qwen3_235b_instruct.invoke([HumanMessage(content=prompt_format)],
                                    config={"callbacks": [opik_tracer]})
    result_list_model_qwen3_235b_instruct.append(rankSelect)
    
import pandas as pd

result_df_model_qwen3_235b_instruct = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_235b_instruct)
result_df_model_qwen3_235b_instruct.rename(columns={"match_index": "match_index_model_qwen3_235b_instruct"}, inplace=True)
result_df_model_qwen3_235b_instruct.rename(columns={"match_reason": "match_reason_model_qwen3_235b_instruct"}, inplace=True)
df = pd.concat([df.reset_index(drop=True),
                    result_df_model_qwen3_235b_instruct.reset_index(drop=True)], axis=1)
df

/var/folders/wt/qx53z0x950jdy1qsklrdghq40000gn/T/ipykernel_78422/1537995823.py:27: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_df_model_qwen3_235b_instruct = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_235b_instruct)


,original_product,top1_candidate,top2_candidate,top3_candidate,cr,match_index_model_qwen_8b,match_reason_model_qwen_8b,match_index_model_qwen3_30b,match_reason_model_qwen3_30b,match_index_model_qwen3_next_80b,match_reason_model_qwen3_next_80b,match_index_model_qwen3_235b_instruct,match_reason_model_qwen3_235b_instruct
0,乌苏 11°P乌苏啤酒罐装 500ml_听,乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）,雪花啤酒 原汁麦8°P啤酒罐装 500ml_听,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,1,首先提取origin_product的商品信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒罐...,1,首先提取origin_product的信息：品牌为'乌苏'，描述为'11°P乌苏啤酒罐装'，...,1,首先提取origin_product信息：品牌为'乌苏'，商品描述为'11°P乌苏啤酒罐装'...,0,首先提取origin_product的信息：品牌为'乌苏'，商品名为'乌苏啤酒罐装'，描述为...
1,元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶,气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,1,首先提取origin_product的信息：品牌为元气森林，商品名为外星人电解质水，描述为0...,1,首先提取origin_product信息：品牌为元气森林，商品描述为0糖0卡青柠口味电解质饮...,1,首先提取origin_product的信息：品牌为'元气森林'，描述为'0糖0卡青柠口味电解...,1,首先提取origin_product的信息：品牌为'元气森林'，商品名为'外星人电解质水'，...
2,统一 老坛酸菜牛肉面 120g_桶,统一 老坛酸菜牛肉面 120g_桶2,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,统一企业茄皇 茄皇鸡蛋面 120g_桶,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,1,首先，提取 origin_product 的信息：品牌为统一，商品名为老坛酸菜牛肉面，规格数...,1,首先提取origin_product的信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格...,1,首先提取origin_product信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，商品描...,0,首先提取origin_product的信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格...
3,旺仔 精选进口乳源原味小馒头 14g_袋,旺仔 小馒头 独立小包散装牛奶原味14g_袋,旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋,,,1,首先提取origin_product的信息：品牌为旺仔，商品名为精选进口乳源原味小馒头，规格...,1,首先对origin_product进行信息提取：品牌为'旺仔'，商品描述为'精选进口乳源原味...,1,首先，提取origin_product信息：品牌为'旺仔'，商品名为'小馒头'，商品描述为'...,1,首先提取origin_product的信息：品牌为'旺仔'，商品名为'小馒头'，商品描述为'...
4,【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖,【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条,【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条,【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条,,1,首先提取origin_product的信息：品牌为'无'，商品名为'简约经典千鸟格围巾'，描...,0,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...,1,首先，提取origin_product的信息：品牌无法提取（无明确品牌名），商品名为'千鸟格...,0,origin_product提取信息：品牌为空，商品名为‘千鸟格围巾’，描述包含‘简约经典、...
5,【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服,珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套,【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套,睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套,,3,首先对origin_product进行信息提取：品牌部分无法识别，商品名称为'珊瑚绒睡衣'，...,1,首先提取origin_product的信息：品牌未明确，商品描述为‘珊瑚绒睡衣’、‘秋冬开衫...,3,首先，提取origin_product信息：品牌未明确提及，商品名为'珊瑚绒睡衣'，描述为'...,0,首先提取origin_product信息：品牌未明确提及，商品名为'珊瑚绒睡衣'，描述为'秋...
6,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2,统一 冰红茶柠檬味茶饮料 1l4瓶,一冰红茶 柠檬味冰红茶 1L_瓶,统一冰红茶 柠檬味冰红茶 1L_瓶,1,首先，提取商品信息：origin_product的品牌是'统一'，商品名称是'青春无极限冰红...,1,首先提取origin_product的信息：品牌为'统一'，商品描述为'青春无极限冰红茶柠檬...,1,首先提取origin_product信息：品牌为'统一'，商品描述为'青春无极限、柠檬味茶饮...,0,首先提取origin_product的信息：品牌为'统一'，商品描述为'青春无极限、柠檬味茶...
7,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶',元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）,,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,1,首先提取origin_product的信息：品牌为元气，商品名为自在水，商品描述为0糖0脂红...,2,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...,2,首先提取origin_product信息：品牌为'元气自在水'（根据规则，品牌位于前部，'元...,1,首先提取origin_product信息：品牌为'元气自在水'，商品名为'自在水'（根据命名...
8,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2',"乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,1,首先提取origin_product的信息：品牌为‘百事食品乐事’，商品名为‘意大利香浓红烩...,3,首先提取origin_product的信息：品牌为'百事食品乐事'（根据规则1，品牌位于信息...,1,首先提取origin_product信息：品牌为'百事食品乐事'，描述为'意大利香浓红烩味原...,3,首先提取origin_product的信息：品牌为'百事食品乐事'，商品名为'原切马铃薯片'...
9,喜力Heineken 11.4°P啤酒 500ml3罐_包,喜力Heineken 11.4°P啤酒 500ml3罐_组,喜力Heineken 11.4°P啤酒 500ml6罐_箱,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,喜力Heineken 11.4°P罐装啤酒 500ml3听罐,1,首先，提取origin_product的信息：品牌为喜力Heineken，商品名为11.4°...,1,首先提取origin_product的信息：品牌为喜力Heineken，商品描述为11.4°...,1,首先提取origin_product信息：品牌为'喜力Heineken'，商品描述为'11....,1,首先提取origin_product信息：品牌为'喜力Heineken'，商品名为'啤酒'，...


In [34]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)
# 迭代调用模型，存储结果到列表
# model_qwen3_next_80b
# model_qwen3_30b
# model_qwen3_max
# model_qwen_8b
# model_glm_9b
# model_qwen3_235b_instruct
result_list_model_qwen3_max = []
for example in examples:
    # 获取元素
    origin_product = example.get("original_product")
    top1_candidate = example.get("top1_candidate")  
    top2_candidate = example.get("top2_candidate")
    top3_candidate = example.get("top3_candidate")
    prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
    rankSelect = model_qwen3_max.invoke([HumanMessage(content=prompt_format)],
                                    config={"callbacks": [opik_tracer]})
    result_list_model_qwen3_max.append(rankSelect)
    
import pandas as pd

result_df_model_qwen3_max = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_max)
result_df_model_qwen3_max.rename(columns={"match_index": "match_index_model_qwen3_max"}, inplace=True)
result_df_model_qwen3_max.rename(columns={"match_reason": "match_reason_model_qwen3_max"}, inplace=True)
df = pd.concat([df.reset_index(drop=True),
                    result_df_model_qwen3_max.reset_index(drop=True)], axis=1)
df

/var/folders/wt/qx53z0x950jdy1qsklrdghq40000gn/T/ipykernel_78422/3692843185.py:27: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_df_model_qwen3_max = pd.DataFrame(obj.dict() for obj in result_list_model_qwen3_max)


,original_product,top1_candidate,top2_candidate,top3_candidate,cr,match_index_model_qwen_8b,match_reason_model_qwen_8b,match_index_model_qwen3_30b,match_reason_model_qwen3_30b,match_index_model_qwen3_next_80b,match_reason_model_qwen3_next_80b,match_index_model_qwen3_235b_instruct,match_reason_model_qwen3_235b_instruct,match_index_model_qwen3_max,match_reason_model_qwen3_max
0,乌苏 11°P乌苏啤酒罐装 500ml_听,乌苏 11°P乌苏啤酒 500ml_听3（新老包装随机发货）,雪花啤酒 原汁麦8°P啤酒罐装 500ml_听,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,乌苏啤酒 11°P夺命大乌苏啤酒 500ml_罐,1,首先提取origin_product的商品信息：品牌为'乌苏'，商品名为'11°P乌苏啤酒罐...,1,首先提取origin_product的信息：品牌为'乌苏'，描述为'11°P乌苏啤酒罐装'，...,1,首先提取origin_product信息：品牌为'乌苏'，商品描述为'11°P乌苏啤酒罐装'...,0,首先提取origin_product的信息：品牌为'乌苏'，商品名为'乌苏啤酒罐装'，描述为...,1,首先提取origin_product信息：品牌为'乌苏'，商品描述为'11°P乌苏啤酒罐装'...
1,元气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,元气森林 外星人电解质水饮料 白桃口味0糖0卡 500ml_瓶,气森林外星人电解质水 0糖0卡青柠口味电解质饮料 500ml_瓶3（新老包装随机发货）,元气森林 外星人电解质水 青柠口味0糖0卡 500ml_瓶,1,首先提取origin_product的信息：品牌为元气森林，商品名为外星人电解质水，描述为0...,1,首先提取origin_product信息：品牌为元气森林，商品描述为0糖0卡青柠口味电解质饮...,1,首先提取origin_product的信息：品牌为'元气森林'，描述为'0糖0卡青柠口味电解...,1,首先提取origin_product的信息：品牌为'元气森林'，商品名为'外星人电解质水'，...,1,首先提取origin_product信息：品牌为'元气森林'，商品名为'外星人电解质水'，描...
2,统一 老坛酸菜牛肉面 120g_桶,统一 老坛酸菜牛肉面 120g_桶2,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,统一企业茄皇 茄皇鸡蛋面 120g_桶,【单桶】统一来一桶老坛酸菜牛肉面桶装速食宵夜食品泡面 120g_桶,1,首先，提取 origin_product 的信息：品牌为统一，商品名为老坛酸菜牛肉面，规格数...,1,首先提取origin_product的信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格...,1,首先提取origin_product信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，商品描...,0,首先提取origin_product的信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格...,0,首先提取origin_product信息：品牌为'统一'，商品名为'老坛酸菜牛肉面'，规格数...
3,旺仔 精选进口乳源原味小馒头 14g_袋,旺仔 小馒头 独立小包散装牛奶原味14g_袋,旺仔 牛奶糖装结婚糖果喜庆原味儿童零食15g_袋,,,1,首先提取origin_product的信息：品牌为旺仔，商品名为精选进口乳源原味小馒头，规格...,1,首先对origin_product进行信息提取：品牌为'旺仔'，商品描述为'精选进口乳源原味...,1,首先，提取origin_product信息：品牌为'旺仔'，商品名为'小馒头'，商品描述为'...,1,首先提取origin_product的信息：品牌为'旺仔'，商品名为'小馒头'，商品描述为'...,1,首先提取origin_product信息：品牌为'旺仔'，商品名为'小馒头'，描述为'精选进...
4,【颜色可选】简约经典千鸟格围巾 1条 女冬季新款韩版氛围感双面学生保暖加厚针织高级感围脖,【复古千鸟格围巾】女冬季 新款韩系百搭学生情侣针织保暖围脖1_条,【蓝棕】格子围巾女冬季2023新款针织菱格千鸟格韩版仿羊绒保暖围脖披肩 _条,【围巾】秋冬韩系围巾新款格子韩版披肩加厚保暖围脖 1条,,1,首先提取origin_product的信息：品牌为'无'，商品名为'简约经典千鸟格围巾'，描...,0,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...,1,首先，提取origin_product的信息：品牌无法提取（无明确品牌名），商品名为'千鸟格...,0,origin_product提取信息：品牌为空，商品名为‘千鸟格围巾’，描述包含‘简约经典、...,0,首先提取origin_product信息：品牌未明确出现（无独立品牌名），商品名为'千鸟格围...
5,【颜色可选】珊瑚绒睡衣 1套 秋冬开衫高颜值女法兰绒长袖睡衣睡裤 加厚保暖可外穿加绒家居服,珊瑚绒睡衣女冬季加厚加绒家居服女法兰绒可外穿保暖套装秋冬款_套,【保暖睡衣】粉色珊瑚绒加绒加厚长袖家居服女水蜜桃毛绒秋冬法兰绒女彩虹花可外穿两件套_套,睡衣冬季女款珊瑚绒加厚保暖长袖宽松可爱草莓法兰绒家居服套装 1套,,3,首先对origin_product进行信息提取：品牌部分无法识别，商品名称为'珊瑚绒睡衣'，...,1,首先提取origin_product的信息：品牌未明确，商品描述为‘珊瑚绒睡衣’、‘秋冬开衫...,3,首先，提取origin_product信息：品牌未明确提及，商品名为'珊瑚绒睡衣'，描述为'...,0,首先提取origin_product信息：品牌未明确提及，商品名为'珊瑚绒睡衣'，描述为'秋...,1,首先提取origin_product信息：品牌无法明确提取（开头为【颜色可选】，无明确品牌名...
6,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶,统一 青春无极限冰红茶柠檬味茶饮料 1000ml_瓶2,统一 冰红茶柠檬味茶饮料 1l4瓶,一冰红茶 柠檬味冰红茶 1L_瓶,统一冰红茶 柠檬味冰红茶 1L_瓶,1,首先，提取商品信息：origin_product的品牌是'统一'，商品名称是'青春无极限冰红...,1,首先提取origin_product的信息：品牌为'统一'，商品描述为'青春无极限冰红茶柠檬...,1,首先提取origin_product信息：品牌为'统一'，商品描述为'青春无极限、柠檬味茶饮...,0,首先提取origin_product的信息：品牌为'统一'，商品描述为'青春无极限、柠檬味茶...,0,首先提取origin_product信息：品牌为'统一'，商品描述为'青春无极限、柠檬味茶饮...
7,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶',元气森林元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶3（新老包装随机发货）,,元气森林 自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,1,首先提取origin_product的信息：品牌为元气，商品名为自在水，商品描述为0糖0脂红...,2,首先对origin_product和各候选商品进行信息提取：\n\n1. origin_pr...,2,首先提取origin_product信息：品牌为'元气自在水'（根据规则，品牌位于前部，'元...,1,首先提取origin_product信息：品牌为'元气自在水'，商品名为'自在水'（根据命名...,2,首先提取origin_product信息：品牌为'元气自在水'，但根据规则，品牌通常位于前部...
8,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋,百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋2',"乐事 意大利香浓红烩味薯片 70g_袋', 规格='70克/袋",乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,乐事 意大利香浓红烩味薯片 休闲食品小吃零食办公室小吃膨化食品 40g_袋,1,首先提取origin_product的信息：品牌为‘百事食品乐事’，商品名为‘意大利香浓红烩...,3,首先提取origin_product的信息：品牌为'百事食品乐事'（根据规则1，品牌位于信息...,1,首先提取origin_product信息：品牌为'百事食品乐事'，描述为'意大利香浓红烩味原...,3,首先提取origin_product的信息：品牌为'百事食品乐事'，商品名为'原切马铃薯片'...,3,首先提取origin_product信息：品牌为'百事食品乐事'（根据规则，品牌通常在前部，...
9,喜力Heineken 11.4°P啤酒 500ml3罐_包,喜力Heineken 11.4°P啤酒 500ml3罐

In [36]:
# 导出结果excel
df.to_excel("llm_rank_result.xlsx", index=False)